# Chainlit Help - 프롬프트 토큰 분석

현재 프로젝트에서 사용하는 프롬프트의 토큰 수를 계산합니다.

참고: [Gemini API 토큰 계산 문서](https://ai.google.dev/gemini-api/docs/tokens?authuser=1&hl=ko&lang=python)


In [18]:
import os
import json
from google import genai
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate

# 환경변수 로드
load_dotenv('app/.env')

# Google AI API 키 설정
api_key = os.getenv('GOOGLE_API_KEY')
if not api_key:
    print("GOOGLE_API_KEY가 설정되지 않았습니다. app/.env 파일을 확인해주세요.")
else:
    client = genai.Client(api_key=api_key)
    print("Google AI 클라이언트 초기화 완료")


Google AI 클라이언트 초기화 완료


In [19]:
# 컨텍스트 파일들 로드
def load_context_files():
    context_files = {
        'codebase': 'app/context/codebase.txt',
        'documentation': 'app/context/documentation.txt', 
        'cookbook': 'app/context/cookbook.txt'
    }
    
    context_content = {}
    
    for name, path in context_files.items():
        try:
            with open(path, 'r', encoding='utf-8') as f:
                content = f.read()
                context_content[name] = content
                print(f"{name}: {len(content):,} 문자")
        except FileNotFoundError:
            print(f"⚠️ {path} 파일을 찾을 수 없습니다.")
            context_content[name] = ""
    
    return context_content

context_content = load_context_files()


codebase: 62,648 문자
documentation: 257,654 문자
cookbook: 44,054 문자


In [31]:
# RAG 프롬프트 템플릿 로드
with open('app/prompts/rag.json', 'r', encoding='utf-8') as f:
    rag_prompt = json.load(f)

print("RAG 프롬프트 설정:")
print(f"모델: {rag_prompt['settings']['model']}")
print(f"온도: {rag_prompt['settings']['temperature']}")
print("\n시스템 프롬프트 템플릿:")
print(rag_prompt['template_messages'][0]['content'][:200] + "...")


RAG 프롬프트 설정:
모델: gemini-2.0-flash
온도: 0.2

시스템 프롬프트 템플릿:
You are a helpful assistant who answers questions related to Chainlit in valid markdown. Keep it short, and if available prefer responding with code. For documentation links, always use fully formed u...


In [32]:
langchain_template = rag_prompt['template_messages'][0]['content'].replace('{{', '{').replace('}}', '}')

# 실제 시스템 프롬프트 구성 (app.py와 동일하게)
system_prompt_template = PromptTemplate(
    input_variables=["codebase", "documentation", "cookbook"],
    template=langchain_template
)

In [33]:
system_prompt_template

PromptTemplate(input_variables=['codebase', 'cookbook', 'documentation'], input_types={}, partial_variables={}, template='You are a helpful assistant who answers questions related to Chainlit in valid markdown. Keep it short, and if available prefer responding with code. For documentation links, always use fully formed urls using this domain: `https://docs.chainlit.io`. Do NOT create APIs, methods or anything else outside of provided context. NEVER hallucinate or extrapolate on the context.\n\nContext:\n<codebase>\n{codebase}\n</codebase>\n<documentation>\n{documentation}\n</documentation>\n<cookbook>\n{cookbook}\n</cookbook>\n\n')

In [34]:
# 템플릿에 컨텍스트 내용 삽입
formatted_system_prompt = system_prompt_template.format_prompt(
    codebase=context_content['codebase'],
    documentation=context_content['documentation'],
    cookbook=context_content['cookbook']
)

In [35]:
formatted_system_prompt

StringPromptValue(text='You are a helpful assistant who answers questions related to Chainlit in valid markdown. Keep it short, and if available prefer responding with code. For documentation links, always use fully formed urls using this domain: `https://docs.chainlit.io`. Do NOT create APIs, methods or anything else outside of provided context. NEVER hallucinate or extrapolate on the context.\n\nContext:\n<codebase>\nDirectory structure:\n└── chainlit-chainlit/\n    └── libs/\n        └── react-client/\n            └── src/\n                ├── context.ts\n                ├── index.ts\n                ├── state.ts\n                ├── useAudio.ts\n                ├── useChatData.ts\n                ├── useChatInteract.ts\n                ├── useChatMessages.ts\n                ├── useChatSession.ts\n                ├── useConfig.ts\n                ├── api/\n                │   ├── index.tsx\n                │   └── hooks/\n                │       ├── api.ts\n                │       

In [37]:
print(f"완성된 시스템 프롬프트 길이: {len(formatted_system_prompt.text):,} 문자")

완성된 시스템 프롬프트 길이: 364,823 문자


In [40]:
# 컨텍스트 토큰 수 계산
if api_key:
    model_name = rag_prompt['settings']['model']  # gemini-2.0-flash
    
    print(f"모델: {model_name}")
    print("=" * 50)
    
    # 전체 시스템 프롬프트
    total_tokens = client.models.count_tokens(model=model_name, contents=[formatted_system_prompt.text])
    print(f"\n 전체 시스템 프롬프트: {total_tokens.total_tokens:,} 토큰")
    
    print("\n" + "=" * 50)
    print("토큰 분석 요약:")
    print(f"- 총합: {total_tokens.total_tokens:,} 토큰")
else:
    print("API 키가 없어서 토큰 계산을 수행할 수 없습니다.")


모델: gemini-2.0-flash

 전체 시스템 프롬프트: 100,081 토큰

토큰 분석 요약:
- 총합: 100,081 토큰
